Softmax回归
===

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.9
IPython 7.8.0

torch 1.2.0


In [2]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

# 1.导入数据集MNIST

In [14]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

random_seed = 123
learning_rate = 0.1
num_epochs = 10
batch_size = 256
num_features = 784
num_classes = 10

train_dataset = datasets.MNIST(root='/Users/JJZHK/data/input/', 
                               train=True, 
                               transform=transforms.ToTensor(),  
                               download=False)

test_dataset = datasets.MNIST(root='/Users/JJZHK/data/input/', 
                              train=False, 
                              transform=transforms.ToTensor(),
                              download=False)


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)


# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])


In [15]:
class SoftmaxRegression(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = torch.nn.Linear(num_features, num_classes)
        
        self.linear.weight.detach().zero_()
        self.linear.bias.detach().zero_()
        
    def forward(self, x):
        logits = self.linear(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

model = SoftmaxRegression(num_features=num_features,
                          num_classes=num_classes)

model.to(device)

##########################
### COST AND OPTIMIZER
##########################

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [16]:
# Manual seed for deterministic data loader
torch.manual_seed(random_seed)


def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    
    for features, targets in data_loader:
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
        
    return correct_pred.float() / num_examples * 100
    

for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        
        # note that the PyTorch implementation of
        # CrossEntropyLoss works with logits, not
        # probabilities
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost))
            
    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/234 | Cost: 2.3026
Epoch: 001/010 | Batch 050/234 | Cost: 0.7941
Epoch: 001/010 | Batch 100/234 | Cost: 0.5651
Epoch: 001/010 | Batch 150/234 | Cost: 0.4603
Epoch: 001/010 | Batch 200/234 | Cost: 0.4822
Epoch: 001/010 training accuracy: 88.04%
Epoch: 002/010 | Batch 000/234 | Cost: 0.4105
Epoch: 002/010 | Batch 050/234 | Cost: 0.4415
Epoch: 002/010 | Batch 100/234 | Cost: 0.4367
Epoch: 002/010 | Batch 150/234 | Cost: 0.4289
Epoch: 002/010 | Batch 200/234 | Cost: 0.3926
Epoch: 002/010 training accuracy: 89.37%
Epoch: 003/010 | Batch 000/234 | Cost: 0.4112
Epoch: 003/010 | Batch 050/234 | Cost: 0.3579
Epoch: 003/010 | Batch 100/234 | Cost: 0.3013
Epoch: 003/010 | Batch 150/234 | Cost: 0.3258
Epoch: 003/010 | Batch 200/234 | Cost: 0.4254
Epoch: 003/010 training accuracy: 89.98%
Epoch: 004/010 | Batch 000/234 | Cost: 0.3988
Epoch: 004/010 | Batch 050/234 | Cost: 0.3690
Epoch: 004/010 | Batch 100/234 | Cost: 0.3459
Epoch: 004/010 | Batch 150/234 | Cost: 0.4030
Epo

In [17]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 91.77%
